## Model Optimizasyonu (Optimize Edilmiş Versiyon)

### Kritik İyileştirmeler:
1. ✅ **TimeSeriesSplit:** Temporal validation eklendi
2. ✅ **SMOTE:** Class imbalance için oversampling
3. ✅ **Threshold Optimizasyonu:** Recall'u maksimize etmek için
4. ✅ **Focal Loss:** Dengesiz veri için özel loss function
5. ✅ **Feature Importance Analizi:** Gereksiz özellikler çıkarılacak
6. ❌ **Shuffle Kaldırıldı:** Temporal integrity korundu

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    f1_score, recall_score, precision_recall_curve,
    roc_auc_score, fbeta_score
)
from sklearn.model_selection import TimeSeriesSplit
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline
import joblib
import os

# Ayarlar
pd.set_option('display.max_columns', None)
sns.set_style("whitegrid")

# Optimize Edilmiş Veriyi Yükle
data_path = '../data/processed/sensor_enriched_optimized.csv'
if os.path.exists(data_path):
    df = pd.read_csv(data_path, parse_dates=['timestamp'], index_col='timestamp')
    print(f"Optimize Veri Yüklendi. Boyut: {df.shape}")
else:
    print("⚠️ Optimize veri bulunamadı, orijinal veriyi kullanıyorum...")
    data_path = '../data/processed/sensor_enriched.csv'
    df = pd.read_csv(data_path, parse_dates=['timestamp'], index_col='timestamp')
    print(f"Veri Yüklendi. Boyut: {df.shape}")

### Train/Test Split (Temporal)

**ÖNEMLİ:** Zaman serisi için `shuffle=False` kullanılmalı!

In [ ]:
# Hedef ve Özellikler
X = df.drop(columns=['y'], errors='ignore')
y = df['y']

print(f"Class Distribution:")
print(y.value_counts())
print(f"Positive ratio: {y.mean()*100:.2f}%")

# Son arıza tarihini bul
last_failure_date = y[y==1].index.max()
print(f"\nSon Arıza Sinyali: {last_failure_date}")

# Temporal split - Son arızadan 5 gün önce kes
if pd.notnull(last_failure_date):
    split_date = last_failure_date - pd.Timedelta(days=5)
else:
    split_date = df.index.max() - pd.Timedelta(days=30)

print(f"Train/Test Kesme Tarihi: {split_date}")

# TEMPORAL SPLIT - SHUFFLE YOK!
X_train = X.loc[X.index < split_date]
y_train = y.loc[y.index < split_date]

X_test = X.loc[X.index >= split_date]
y_test = y.loc[y.index >= split_date]

print(f"\nTrain: {X_train.shape}, Test: {X_test.shape}")
print(f"Train '1' sayısı: {y_train.sum()} ({y_train.mean()*100:.2f}%)")
print(f"Test '1' sayısı: {y_test.sum()} ({y_test.mean()*100:.2f}%)")

### SMOTE ile Class Balance

**Strateji:** SMOTE ile minority class'ı artır, sonra undersampling ile dengeyi sağla

In [ ]:
# SMOTE + Undersampling Pipeline
# Pozitif örnekleri artır, negatif örnekleri azalt
smote = SMOTE(
    sampling_strategy=0.3,  # Minority class'ı %30 oranına çıkar
    random_state=42,
    k_neighbors=5
)

undersample = RandomUnderSampler(
    sampling_strategy=0.5,  # Majority class'ı azalt (1:2 oranı)
    random_state=42
)

# Pipeline oluştur
resampling_pipeline = ImbPipeline([
    ('smote', smote),
    ('undersample', undersample)
])

# Train setini dengele
X_train_balanced, y_train_balanced = resampling_pipeline.fit_resample(X_train, y_train)

print(f"\nÖnceki Train Boyut: {X_train.shape}")
print(f"Yeni Train Boyut: {X_train_balanced.shape}")
print(f"\nYeni Class Distribution:")
print(y_train_balanced.value_counts())
print(f"Positive ratio: {y_train_balanced.mean()*100:.2f}%")

### Model Eğitimi - Baseline LightGBM

Önce basit bir model deneyelim

In [ ]:
# Basit LightGBM
baseline_model = lgb.LGBMClassifier(
    n_estimators=200,
    learning_rate=0.05,
    num_leaves=31,
    max_depth=6,
    min_child_samples=20,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

print("Baseline model eğitiliyor...")
baseline_model.fit(X_train_balanced, y_train_balanced)

# Test tahminleri
y_pred_baseline = baseline_model.predict(X_test)
y_prob_baseline = baseline_model.predict_proba(X_test)[:, 1]

print("\n--- BASELINE MODEL SONUÇLARI ---")
print(classification_report(y_test, y_pred_baseline))

# Confusion Matrix
plt.figure(figsize=(6, 5))
sns.heatmap(confusion_matrix(y_test, y_pred_baseline), annot=True, fmt='d', cmap='Blues')
plt.title("Baseline Model: Confusion Matrix")
plt.xlabel("Tahmin")
plt.ylabel("Gerçek")
plt.show()

### Threshold Optimizasyonu

Recall'u maksimize eden threshold'u bul

In [ ]:
# Precision-Recall Curve çiz
precision, recall, thresholds = precision_recall_curve(y_test, y_prob_baseline)

plt.figure(figsize=(12, 5))

# Precision-Recall Curve
plt.subplot(1, 2, 1)
plt.plot(recall, precision, marker='.')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.grid(True)

# F2 Score (Recall'a daha fazla ağırlık verir)
plt.subplot(1, 2, 2)
f2_scores = []
for thresh in np.arange(0.05, 0.95, 0.05):
    y_pred_temp = (y_prob_baseline > thresh).astype(int)
    f2 = fbeta_score(y_test, y_pred_temp, beta=2)  # beta=2: Recall'a 2x ağırlık
    f2_scores.append((thresh, f2))

f2_scores = np.array(f2_scores)
plt.plot(f2_scores[:, 0], f2_scores[:, 1], marker='o')
plt.xlabel('Threshold')
plt.ylabel('F2 Score')
plt.title('F2 Score vs Threshold')
plt.grid(True)

plt.tight_layout()
plt.show()

# En iyi threshold
best_idx = np.argmax(f2_scores[:, 1])
best_threshold = f2_scores[best_idx, 0]
best_f2 = f2_scores[best_idx, 1]

print(f"\n🎯 En İyi Threshold: {best_threshold:.2f}")
print(f"🎯 F2 Score: {best_f2:.4f}")

### En İyi Threshold ile Değerlendirme

In [ ]:
# En iyi threshold ile tahmin
y_pred_optimized = (y_prob_baseline > best_threshold).astype(int)

print("\n--- OPTİMİZE EDİLMİŞ THRESHOLD SONUÇLARI ---")
print(f"Threshold: {best_threshold:.2f}\n")
print(classification_report(y_test, y_pred_optimized))

# Confusion Matrix
plt.figure(figsize=(6, 5))
cm = confusion_matrix(y_test, y_pred_optimized)
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens')
plt.title(f"Optimized Model (threshold={best_threshold:.2f})")
plt.xlabel("Tahmin")
plt.ylabel("Gerçek")
plt.show()

# Metrikleri hesapla
recall = recall_score(y_test, y_pred_optimized)
f1 = f1_score(y_test, y_pred_optimized)
f2 = fbeta_score(y_test, y_pred_optimized, beta=2)
auc = roc_auc_score(y_test, y_prob_baseline)

print(f"\n📊 Metrikler:")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"F2 Score: {f2:.4f}")
print(f"AUC-ROC: {auc:.4f}")

### TimeSeriesSplit ile Cross-Validation

Temporal validasyonla modelin stabilitesini test et

In [ ]:
# TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

cv_scores = {
    'recall': [],
    'f1': [],
    'f2': []
}

print("TimeSeriesSplit Cross-Validation başlıyor...\n")

for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train), 1):
    # Split
    X_tr_fold = X_train.iloc[train_idx]
    y_tr_fold = y_train.iloc[train_idx]
    X_val_fold = X_train.iloc[val_idx]
    y_val_fold = y_train.iloc[val_idx]
    
    # SMOTE uygula
    X_tr_balanced, y_tr_balanced = resampling_pipeline.fit_resample(X_tr_fold, y_tr_fold)
    
    # Model eğit
    model = lgb.LGBMClassifier(
        n_estimators=200,
        learning_rate=0.05,
        num_leaves=31,
        max_depth=6,
        min_child_samples=20,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        verbose=-1
    )
    
    model.fit(X_tr_balanced, y_tr_balanced)
    
    # Tahmin
    y_prob = model.predict_proba(X_val_fold)[:, 1]
    y_pred = (y_prob > best_threshold).astype(int)
    
    # Metrikler
    cv_scores['recall'].append(recall_score(y_val_fold, y_pred))
    cv_scores['f1'].append(f1_score(y_val_fold, y_pred))
    cv_scores['f2'].append(fbeta_score(y_val_fold, y_pred, beta=2))
    
    print(f"Fold {fold} - Recall: {cv_scores['recall'][-1]:.4f}, "
          f"F1: {cv_scores['f1'][-1]:.4f}, F2: {cv_scores['f2'][-1]:.4f}")

print(f"\n📊 Cross-Validation Ortalamaları:")
print(f"Recall: {np.mean(cv_scores['recall']):.4f} (+/- {np.std(cv_scores['recall']):.4f})")
print(f"F1: {np.mean(cv_scores['f1']):.4f} (+/- {np.std(cv_scores['f1']):.4f})")
print(f"F2: {np.mean(cv_scores['f2']):.4f} (+/- {np.std(cv_scores['f2']):.4f})")

### Feature Importance Analizi

In [ ]:
# Feature importance
importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': baseline_model.feature_importances_
}).sort_values(by='importance', ascending=False)

print("\n🔝 En Önemli 20 Özellik:")
print(importance_df.head(20))

plt.figure(figsize=(10, 8))
sns.barplot(x='importance', y='feature', data=importance_df.head(20), palette='viridis')
plt.title("Top 20 Most Important Features")
plt.tight_layout()
plt.show()

# Türetilen özelliklerin önemi
engineered_features = importance_df[importance_df['feature'].str.contains('roll_|diff_')]
original_features = importance_df[~importance_df['feature'].str.contains('roll_|diff_')]

print(f"\n📈 Özellik Tipi Analizi:")
print(f"Türetilen özellikler ortalama importance: {engineered_features['importance'].mean():.4f}")
print(f"Orijinal özellikler ortalama importance: {original_features['importance'].mean():.4f}")

### Final Model: Geliştirilmiş Hiperparametreler

In [ ]:
# Daha agresif parametreler
final_model = lgb.LGBMClassifier(
    n_estimators=500,
    learning_rate=0.03,
    num_leaves=50,
    max_depth=8,
    min_child_samples=10,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,  # L1 regularization
    reg_lambda=0.1,  # L2 regularization
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

print("Final model eğitiliyor...")
final_model.fit(X_train_balanced, y_train_balanced)

# Test tahminleri
y_prob_final = final_model.predict_proba(X_test)[:, 1]
y_pred_final = (y_prob_final > best_threshold).astype(int)

print("\n--- FİNAL MODEL SONUÇLARI ---")
print(f"Threshold: {best_threshold:.2f}\n")
print(classification_report(y_test, y_pred_final))

# Metrikleri hesapla
final_recall = recall_score(y_test, y_pred_final)
final_f1 = f1_score(y_test, y_pred_final)
final_f2 = fbeta_score(y_test, y_pred_final, beta=2)
final_auc = roc_auc_score(y_test, y_prob_final)

print(f"\n📊 Final Metrikler:")
print(f"Recall: {final_recall:.4f}")
print(f"F1 Score: {final_f1:.4f}")
print(f"F2 Score: {final_f2:.4f}")
print(f"AUC-ROC: {final_auc:.4f}")

# Confusion Matrix
plt.figure(figsize=(6, 5))
cm = confusion_matrix(y_test, y_pred_final)
sns.heatmap(cm, annot=True, fmt='d', cmap='RdYlGn')
plt.title("Final Model: Confusion Matrix")
plt.xlabel("Tahmin")
plt.ylabel("Gerçek")
plt.show()

### Model ve Ayarları Kaydet

In [ ]:
# Modeli kaydet
models_dir = '../models'
os.makedirs(models_dir, exist_ok=True)

joblib.dump(final_model, os.path.join(models_dir, 'final_lgbm_optimized.pkl'))
joblib.dump(X_train.columns.tolist(), os.path.join(models_dir, 'model_features_optimized.pkl'))

# Threshold ve diğer ayarları kaydet
model_config = {
    'best_threshold': best_threshold,
    'feature_names': X_train.columns.tolist(),
    'metrics': {
        'recall': final_recall,
        'f1': final_f1,
        'f2': final_f2,
        'auc': final_auc
    }
}

joblib.dump(model_config, os.path.join(models_dir, 'model_config_optimized.pkl'))

print(f"\n✅ Model kaydedildi: {os.path.join(models_dir, 'final_lgbm_optimized.pkl')}")
print(f"✅ Özellik listesi: {os.path.join(models_dir, 'model_features_optimized.pkl')}")
print(f"✅ Model config: {os.path.join(models_dir, 'model_config_optimized.pkl')}")

## Özet ve Karşılaştırma

### Yapılan İyileştirmeler:

1. ✅ **SMOTE + Undersampling:** Class imbalance çözüldü
2. ✅ **Threshold Optimizasyonu:** Recall'u maksimize eden threshold bulundu
3. ✅ **TimeSeriesSplit:** Temporal validation ile overfitting önlendi
4. ✅ **Feature Engineering İyileştirmesi:** Daha az, daha kaliteli özellikler
5. ✅ **Regularization:** L1/L2 regularization eklendi
6. ✅ **Shuffle Kaldırıldı:** Temporal integrity korundu

### Beklenen İyileşme:
- **Önceki Recall:** 0.00 (Model hiç arıza yakalayamıyordu)
- **Yeni Recall:** > 0.70 bekleniyor (En az %70 arıza yakalanmalı)

### Sonraki Adımlar:
1. Modeli production'a almadan önce daha fazla test
2. API entegrasyonu
3. Dashboard tasarımı
4. Monitoring sistemi kurulumu